In [38]:
import os
import csv
from itertools import combinations
import fireducks.pandas as pd
from sklearn.model_selection import train_test_split
import evaluate
from sentence_transformers import CrossEncoder, SentenceTransformer
import numpy as np
import joblib
from copy import deepcopy

In [13]:
rouge = evaluate.load("rouge")

In [11]:
model = CrossEncoder("BAAI/bge-reranker-v2-m3")

In [35]:
test_path = os.path.expanduser('~/definitions/ax_rus_2.tsv.gz')
test = pd.read_csv(test_path, sep='\t', quoting=csv.QUOTE_NONE)
test.head()

,usage_id,word,orth,sense_id,gloss,Context,indices_target_token,date,period,Real_Contexts,Generated_Definition
0,test_ru_0,мёрзлый,мёрзлый,merzlyj_khrrgztit4U,"разг., устар. очень холодный; ледяной","Мерзлая земля скажется, только руку приложи От...",NaN,old,old,"Мерзлая земля скажется, только руку приложи От...","перен., разг. сильно влажный, мокрый"
1,test_ru_1,мёрзлый,мёрзлый,merzlyj_Webtz03k0N8,"прост. озябший, окоченевший",Пантелей Прокофьевич сунул Григорию мёрзлую ру...,NaN,new,new,Пантелей Прокофьевич сунул Григорию мёрзлую ру...,"перен., разг. мокрый, влажный"
2,test_ru_2,мёрзлый,мёрзлый,merzlyj_g74j6SCAoAc,покрытый ледяной коркой; обледенелый,"Между домом и рельсами, за широкой мёрзлой луж...",NaN,new,new,"Между домом и рельсами, за широкой мёрзлой луж...","перен., разг. мокрый, влажный"
3,test_ru_3,мёрзлый,мёрзлый,merzlyj_KzCCL3oBy_Y,устар. погибший от холода; замёрзший,"— Да, холодно, должно быть. На полу мёрзлые та...",NaN,new,new,"— Да, холодно, должно быть. На полу мёрзлые та...","перен., разг. мокрый, влажный"
4,test_ru_4,мёрзлый,мёрзлый,merzlyj_khrrgztit4U,"разг., устар. очень холодный; ледяной","Когда мосты были прорваны, безоружные солдаты,...",NaN,new,new,"Когда мосты были прорваны, безоружные солдаты,...","перен., разг. мокрый, влажный"


In [8]:
clf = joblib.load('mt0-fi-ru.joblib')

In [46]:
test_our = pd.read_csv(
    "../../axolotl24_shared_task/data/test/axolotl.test.ru.tsv", sep='\t', quoting=csv.QUOTE_NONE)
test_our['Generated_Definition'] = test['Generated_Definition']

In [58]:

for word in test.word.unique():
    this_word = test[test.word==word]
    new = this_word[this_word.period == 'new']
    olds = this_word[this_word.period == 'old']
    if olds.shape[0] > 0:
        for defgen in new.Generated_Definition:
            preds = [old_gloss+'\n' for old_gloss in olds.gloss]
            refs = [defgen+'\n' for _ in olds.gloss]
            rouges = rouge.compute(
    predictions=preds, references=refs,
                tokenizer=lambda y: y.split(), use_aggregator=False,
    )
            sims = model.predict([(old_gloss, defgen) for old_gloss in olds.gloss])
            features = pd.DataFrame({'sims': sims,
                                     'rouge1': rouges['rouge1'],
                                     'rouge2': rouges['rouge2'],
                                     'rougeL': rouges['rougeL'],
                                     'rougeLsum': rouges['rougeLsum']})
            predictions = clf.predict(features)
            ones = np.nonzero(predictions)[0]
            if ones.shape[0] > 0:
                if ones.shape[0] == 1:
                    old_sense = olds.reset_index().iloc[ones[0]].sense_id
                elif ones.shape[0] > 1:
                    predictions = clf.predict_proba(features)
                    highest = np.argmax(predictions[:,1])
                    old_sense = olds.reset_index().iloc[highest].sense_id
                test_our.loc[(test_our.word==word) & (test_our.Generated_Definition == defgen)].sense_id = old_sense

In [ ]:
test_our.loc[(test_our.word=='мёрзлый') & (test_our.Generated_Definition == defgen)].sense_id = old_sense

In [61]:
for our, gold in zip(test_our[test_our.period == 'new'].sense_id, test[test.period == 'new'].sense_id):
    if our != gold:
        print((our, gold))

(None, 'merzlyj_Webtz03k0N8')
(None, 'merzlyj_g74j6SCAoAc')
(None, 'merzlyj_KzCCL3oBy_Y')
(None, 'merzlyj_khrrgztit4U')
(None, 'merzlyj_khrrgztit4U')
(None, 'merzlyj_g74j6SCAoAc')
(None, 'merzlyj_B3Ndzk2__5o')
(None, 'merzlyj_khrrgztit4U')
(None, 'merzlyj_khrrgztit4U')
(None, 'merzlyj_Webtz03k0N8')
(None, 'merzlyj_g74j6SCAoAc')
(None, 'merzlyj_khrrgztit4U')
(None, 'element_p7W7YI7KDl4')
(None, 'element_O-Xj64Jj8io')
(None, 'element_mip-D10cqu0')
(None, 'element_O-Xj64Jj8io')
(None, 'element_s83wk-Hgdp4')
(None, 'element_p7W7YI7KDl4')
(None, 'element_O-Xj64Jj8io')
(None, 'element_O-Xj64Jj8io')
(None, 'element_O-Xj64Jj8io')
(None, 'kabak_YqFV8T7CgM8')
(None, 'kabak_7HYPew2Q9vM')
(None, 'kabak_hO7-f2Xxaic')
(None, 'kabak_7HYPew2Q9vM')
(None, 'kabak_WwCw6L9v9ek')
(None, 'kabak_hO7-f2Xxaic')
(None, 'kabak_YqFV8T7CgM8')
(None, 'kabak_WwCw6L9v9ek')
(None, 'kabak_WwCw6L9v9ek')
(None, 'kabak_hO7-f2Xxaic')
(None, 'kabak_WwCw6L9v9ek')
(None, 'trubach_gqKwuDGrbd4')
(None, 'trubach_UkbiF9TOfxA')
(N

In [60]:
test_our[test_our.period == 'new'].sense_id.unique()

array([None], dtype=object)

In [48]:
from sklearn.metrics import adjusted_rand_score
from collections import Counter
from leven import levenshtein

In [104]:
def find_merges(df):
    targ_words = df.word.unique()

    # Finding definitions similar to the most frequent one (paraphrases) for each targ_word:
    mappings = {targ_word: {} for targ_word in targ_words}

    for targ_word in targ_words:
        definitions = Counter(df[df.word == targ_word].Generated_Definition.dropna()).most_common()
        if definitions:
        # definitions = sorted(definitions)  # can be commented to out to start from most frequent
            sim_cache = {}
            def2compare = None
            for nr, source in enumerate(definitions):
                cand = source[0]
                if len(cand.split()) >= 4 and cand not in mappings[targ_word]:
                    def2compare = cand
                    mapped = 0
                    for definition in definitions:
                        definition_text = definition[0]
                        if definition_text != def2compare and \
                                definition_text not in mappings[targ_word]:
                            if (definition_text, def2compare) in sim_cache:
                                distance = sim_cache[(definition_text, def2compare)]
                            else:
                                distance = levenshtein(definition_text, def2compare)
                                sim_cache[(definition_text, def2compare)] = distance
                                sim_cache[(def2compare, definition_text)] = distance
                            if distance < 10:
                                mappings[targ_word][definition_text] = def2compare
                                mapped += 1

                if def2compare:
                    break

    return mappings, targ_words

In [105]:
mappings, targ_words = find_merges(train_new)

In [106]:
mappings

{'могильник': {},
 'монитор': {},
 'мороз': {'устар. холодная погодa': 'устар. холодное время года'},
 'мундштук': {'часть трубки, соединяющая рот с остальной частью папиросы': 'часть трубки, соединяющая рот с остальной частью трубки'},
 'муж': {},
 'мужчина': {'лингв., разг. человек, принадлежающий мужчине [ 1 ]': 'лингв., устар. человек, принадлежающий к мужчине [ 1 ]'},
 'мурман': {},
 'мясо': {},
 'нос': {},
 'нагулять': {},
 'надёжный': {'обладающий высоким уровнем надежности, стойкости': 'обладающий высоким уровнем надежности, стабильности'},
 'надой': {},
 'назначить': {},
 'наиграть': {},
 'наносить': {},
 'наново': {},
 'наносный': {},
 'нараспашку': {},
 'номер': {},
 'нора': {},
 'нота': {},
 'набрать': {},
 'наверх': {},
 'находить': {},
 'натирать': {},
 'натурный': {'настоящий, истиный, действующий на деле': 'настоящий, живой, действующий на деле'},
 'нейтральный': {},
 'неподвижный': {},
 'нива': {},
 'низменный': {},
 'ник': {},
 'обводить': {'обрисовываться, рисовать п

In [107]:
targ_words

array(['могильник', 'монитор', 'мороз', 'мундштук', 'муж', 'мужчина',
       'мурман', 'мясо', 'нос', 'нагулять', 'надёжный', 'надой',
       'назначить', 'наиграть', 'наносить', 'наново', 'наносный',
       'нараспашку', 'номер', 'нора', 'нота', 'набрать', 'наверх',
       'находить', 'натирать', 'натурный', 'нейтральный', 'неподвижный',
       'нива', 'низменный', 'ник', 'обводить', 'облако', 'областной',
       'обмывать', 'обрывать', 'обух', 'отступать', 'отдать', 'отрубить',
       'отчаянный', 'оговорить', 'округлить', 'общий', 'опадать',
       'оптика', 'опутать', 'лента', 'летун', 'лещ', 'лизать', 'линия',
       'лист', 'литовка', 'логовище', 'ложе', 'лох', 'луковица', 'падла',
       'паз', 'папка', 'парафраз', 'парк', 'паровоз', 'партия', 'пачка',
       'перевёртыш', 'передок', 'передовой', 'пересыпать', 'перетереть',
       'перо', 'петрушка', 'пионер', 'платформа', 'плотный', 'поболтать',
       'побывать', 'погасить', 'пул', 'пульс', 'пустить', 'путь',
       'пуховый',

In [108]:
new_defs = []
for word in train_new.word.unique():
    word_map = mappings[word]
    this_word = train_new[train_new.word==word]
    for defi in this_word.Generated_Definition:
        new = word_map.get(defi, defi)
        new_defs.append(defi)


In [109]:
ari = adjusted_rand_score(train_new.sense_id, new_defs)
ari

0.11673766924128653